# Improve 6 : AutoEncoder

## Best Version

```python
base_models = []
base_models.append(('lr', LogisticRegression(max_iter=10000)))  # Increased max_iter
base_models.append(('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')))  # Adjusted parameters for XGBClassifier
base_models.append(('rf', RandomForestClassifier()))
base_models.append(('svm', SVC()))

meta_model = LogisticRegression(max_iter=10000)  # Increased max_iter for meta model as well

clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)
```
- crosss validaion score : 0.4800848691221959
- Leader Board score : 0.34316

## 1. Import library

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
import numpy as np
import re

## 2. Load Data 

In [2]:
train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')

## 3. Data PreProcessing

### 3-1. 모든 행이 WT인 컬럼 제거 (train, test 모두)

In [3]:
# 두 데이터프레임에서 모든 값이 'WT'인 컬럼을 각각 찾음
columns_all_wt_train = train.columns[(train == 'WT').all(axis=0)]
columns_all_wt_test = test.columns[(test == 'WT').all(axis=0)]

# train과 test에서 모든 값이 'WT'인 컬럼의 합집합 찾기
union_columns = columns_all_wt_train.union(columns_all_wt_test)

# 합집합에 해당하는 컬럼을 train과 test에서 삭제
train = train.drop(columns=union_columns)
test = test.drop(columns=union_columns)

# 결과 확인
# print("모든 값이 'WT'인 컬럼을 삭제한 train 데이터프레임:")
# print(train.head())
# print("모든 값이 'WT'인 컬럼을 삭제한 test 데이터프레임:")
# print(test.head())


### 3-2. 표기 규칙 수정 (47s)

In [4]:
# 돌연변이 표기 수정 규칙 적용 함수
def process_mutations(mutation_string):
    if pd.isnull(mutation_string):
        return None  # 결측치 처리

    # 여러 개의 변이를 ',' 또는 ' ' 로 나누어 처리
    mutations = mutation_string.split(',')
    processed_mutations = []

    for mutation in mutations:
        mutation = mutation.strip()
        
        # 다중 변이 패턴: '711_712FL>FL' -> 'F711F L712L'로 수정
        multi_match = re.match(r'^(\d+_\d+)([A-Z]+)>([A-Z*]+)$', mutation)
        if multi_match:
            positions = multi_match.group(1)  # '197_198'
            from_aa_seq = multi_match.group(2)  # 'YQ'
            to_aa_seq = multi_match.group(3)    # '**' 또는 특정 아미노산 코드

            pos_list = positions.split('_')
            new_mutation = []
            for i, (from_aa, to_aa) in enumerate(zip(from_aa_seq, to_aa_seq)):
                # '*'와 같은 특수한 문자가 들어오면 그대로 유지
                if to_aa == '*':
                    new_mutation.append(f"{from_aa}{pos_list[i]}*")
                else:
                    new_mutation.append(f"{from_aa}{pos_list[i]}{to_aa}")
            processed_mutations.append(' '.join(new_mutation))

        # '*' 돌연변이 패턴은 그대로 추가
        elif '*' in mutation:
            processed_mutations.append(mutation)
        
        # 기본적인 패턴: 그대로 추가
        else:
            processed_mutations.append(mutation)

    # 중복 제거 후 반환
    unique_mutations = list(dict.fromkeys(processed_mutations))  # 중복 제거
    return ', '.join(unique_mutations)

In [5]:
train_processed = train.applymap(process_mutations)

In [6]:
test_processed = test.applymap(process_mutations)

### 3-3. misense driver 적용 (1m 26s)

In [7]:
misense_driver = pd.read_csv('./open/missense_driver.csv', sep='\t')

In [8]:
misense_driver['Gene name'] = misense_driver['Gene name'].str.split('_').str[0]

In [9]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [10]:
misense_driver = misense_driver[misense_driver['Gene name'].isin(yes)]

In [11]:
misense_driver_dict = misense_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [12]:
# 다중 돌연변이를 처리하는 함수
def update_misense_driver(train, misense_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_driver_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 30으로 교체
                processed_mutations = [str(30) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [13]:
# 함수 적용
train_processed2 = update_misense_driver(train_processed, misense_driver_dict)
test_processed2 = update_misense_driver(test_processed, misense_driver_dict)

In [14]:
train_processed2[train_processed2['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1758,TRAIN_1758,STES,30,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,30,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,A1913D,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [15]:
test_processed2[test_processed2['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,30,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### 3-4. missense passenger 적용 (2m 44s)

In [16]:
misense_passenger = pd.read_csv('./open/missense_passenger.csv', sep='\t')

In [17]:
misense_passenger['Gene name'] = misense_passenger['Gene name'].str.split('_').str[0]

In [18]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [19]:
misense_passenger = misense_passenger[misense_passenger['Gene name'].isin(yes)]

In [20]:
misense_passenger_dict = misense_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [21]:
def update_misense_passenger(train, misense_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_passenger_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(0) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train

In [22]:
train_processed3=update_misense_passenger(train_processed2,misense_passenger_dict)
test_processed3=update_misense_passenger(test_processed2,misense_passenger_dict)

In [23]:
train_processed3[train_processed3['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,E291E,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2969,TRAIN_2969,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2984,TRAIN_2984,COAD,0,WT,WT,0,S2192N F1854F K376R,WT,WT,V1698V E1295* S1071L,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [24]:
test_processed3[test_processed3['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT


### 3-5. silent driver 적용 (51s)

In [25]:
silent_driver = pd.read_csv('./open/silent_driver.csv', sep='\t')

In [26]:
silent_driver['Gene name'] = silent_driver['Gene name'].str.split('_').str[0]

In [27]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [28]:
silent_driver = silent_driver[silent_driver['Gene name'].isin(yes)]

In [29]:
silent_driver_dict = silent_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [30]:
def update_silent_driver(train, silent_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_driver_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(30) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [31]:
train_processed4 = update_silent_driver(train_processed3,silent_driver_dict)
test_processed4 = update_silent_driver(test_processed3,silent_driver_dict)

In [32]:
train_processed4[train_processed4['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
594,TRAIN_0594,CESC,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1758,TRAIN_1758,STES,30,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,30,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [33]:
test_processed4[test_processed4['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,30,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### 3-6. silent passenger 적용 (2m 42s)

In [34]:
silent_passenger = pd.read_csv('./open/silent_passenger.csv', sep='\t')

In [35]:
silent_passenger['Gene name'] = silent_passenger['Gene name'].str.split('_').str[0]

In [36]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [37]:
silent_passenger = silent_passenger[silent_passenger['Gene name'].isin(yes)]

In [38]:
silent_passenger_dict = silent_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [39]:
def update_silent_passenger(train, silent_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_passenger_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(0) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [40]:
train_processed5 = update_silent_passenger(train_processed4,silent_passenger_dict)
test_processed5 = update_silent_passenger(test_processed4,silent_passenger_dict)

In [41]:
train_processed5[train_processed5['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1502,TRAIN_1502,COAD,0,WT,WT,0,WT,WT,WT,V1774V T311T,...,K162T S664S A779V T1526M L1806V R2115*,WT,WT,WT,Q1572Q H730Q I688I,WT,0,WT,WT,WT
1606,TRAIN_1606,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,0,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [42]:
test_processed5[test_processed5['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1337,TEST_1337,0,WT,WT,WT,WT,WT,WT,WT,WT,...,K1345*,WT,WT,WT,WT,WT,WT,WT,WT,A430E
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT
2214,TEST_2214,0,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


### 3-7. amino acid 함수 정의 + train,test set 적용 (1m 26s)

In [43]:
#aminoacid version3
# 변이 유형을 분류하는 함수 정의
amino_acid_properties = {
    'A': 'nonpolar',    # Alanine
    'R': 'positive',    # Arginine
    'N': 'polar',       # Asparagine
    'D': 'negative',    # Aspartic Acid
    'C': 'polar',       # Cysteine
    'Q': 'polar',       # Glutamine
    'E': 'negative',    # Glutamic Acid
    'G': 'nonpolar',    # Glycine
    'H': 'positive',    # Histidine
    'I': 'nonpolar',    # Isoleucine
    'L': 'nonpolar',    # Leucine
    'K': 'positive',    # Lysine
    'M': 'nonpolar',    # Methionine
    'F': 'aromatic',    # Phenylalanine
    'P': 'nonpolar',    # Proline
    'S': 'polar',       # Serine
    'T': 'polar',       # Threonine
    'W': 'aromatic',    # Tryptophan
    'Y': 'aromatic',    # Tyrosine
    'V': 'nonpolar',    # Valine
}

def classify_mutation(mutation):
    # 결측치 처리
    if pd.isnull(mutation):
        return None  # 또는 특정 코드로 지정 가능

    # 숫자인 경우 또는 숫자로 변환 가능한 경우 그대로 반환
    try:
        mutation = float(mutation)  # 숫자로 변환 시도
        return int(mutation) if mutation.is_integer() else mutation  # 정수형이면 int, 실수형이면 그대로 반환
    except ValueError:
        pass  # 숫자가 아닌 경우 문자열 처리로 넘어감

    mutation = str(mutation).strip()

    # WT 체크
    if mutation == 'WT':
        return 0  # WT (Wild Type)

    # 프레임시프트 돌연변이 체크 ('fs' 포함)
    if 'fs' in mutation:
        return 30  # 프레임시프트 돌연변이

    # 중단 돌연변이 체크 ('*' 포함)
    if '*' in mutation:
        return 30  # 중단 돌연변이

    # 삭제 돌연변이 체크 ('del' 포함)
    if 'del' in mutation:
        return 30  # 삭제 돌연변이 (Deletion mutation)

    # 단일 아미노산 변이 패턴 매칭 (예: 'R496Q', 'L1700L')
    match = re.match(r'^([A-Z])(\d*)([A-Z])$', mutation)
    if match:
        from_aa = match.group(1)  # 원래 아미노산
        position = match.group(2) # 위치 (사용하지 않음)
        to_aa = match.group(3)    # 변이된 아미노산

        # 침묵 돌연변이 체크 (아미노산이 동일한 경우)
        if from_aa == to_aa:
            return 0 # 침묵 돌연변이

        # 아미노산 성질 가져오기
        from_property = amino_acid_properties.get(from_aa)
        to_property = amino_acid_properties.get(to_aa)

        # 아미노산 코드가 유효한지 확인
        if from_property is None or to_property is None:
            return 7  # 알 수 없는 아미노산 코드

        # 보존적 돌연변이 체크 (아미노산 성질이 동일한 경우)
        if from_property == to_property:
            return 2  # 보존적 돌연변이
        else:
            return 3  # 비보존적 돌연변이
    else:
        # 패턴 매칭 실패한 경우
        return 7  # 매칭 실패한 경우 7 반환

# 다중 치환을 처리하는 함수 정의
def classify_multiple_mutations(mutation_string):
    # 결측치 처리
    if pd.isnull(mutation_string):
        return None  # 또는 특정 코드로 지정 가능

    # 변이 문자열을 공백으로 분리
    mutations = set(str(mutation_string).strip().split())

    labels = []
    for mutation in mutations:
        # 숫자인 경우 또는 숫자로 변환 가능한 경우 그대로 유지
        try:
            mutation = float(mutation)  # 숫자로 변환 시도
            mutation = int(mutation) if mutation.is_integer() else mutation  # 정수형이면 int, 실수형이면 그대로 유지
            labels.append(mutation)
        except ValueError:
            # 숫자가 아닌 경우 기존 로직 사용
            label = classify_mutation(mutation)
            if label is not None:
                labels.append(label)

    if labels:
        # 합계 반환
        return sum(labels)
    else:
        return None  # 또는 특정 코드로 지정 가능


In [44]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train_processed5.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    train_processed5[col] = train_processed5[col].apply(classify_multiple_mutations)


In [45]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in test_processed5.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    test_processed5[col] = test_processed5[col].apply(classify_multiple_mutations)


### 3-8. CSV 저장 & 불러오기
- 함수 적용 시간이 오래걸리기때문에 여기까지 trian을 csv로 저장 후 불러내기 해서 사용
- 저장한 train파일에 컬럼(유전자) 수는 원래 유전자수(4865)에서 모든 행이 'WT'이 제거됨 (4226열)
- 저장된 파일은 전처리가 끝난 상태 -> 모델만 수정해가면서 train,test 불러와서 반복해서 돌리기


In [46]:
#4개의 csv파일함수적용 후 'PS_mapping_ver3.csv'로 저장하기 ( 최초 1회만 실행)
train_processed5.to_csv('./open/train_preprocessed.csv')
test_processed5.to_csv('./open/test_preprocessed.csv')

In [57]:
train = pd.read_csv('./open/train_preprocessed.csv')
train = train.iloc[:,1:]
test = pd.read_csv('./open/test_preprocessed.csv')
test = test.iloc[:,1:]

### 3-9. Target Encoding

In [48]:
from sklearn.preprocessing import LabelEncoder
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

## 4. Model definition

In [51]:
#amino acid version3 + basic model
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 1. 데이터 준비
# 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS를 제외한 모든 열)
y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# 2. 학습 세트와 테스트 세트로 데이터 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Base models with increased max_iter for LogisticRegression
base_models = []
base_models.append(('lr', LogisticRegression(random_state=42, max_iter=10000)))  # Increased max_iter
base_models.append(('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1)))  # Adjusted parameters for XGBClassifier
base_models.append(('rf', RandomForestClassifier(random_state=42, n_jobs=-1)))
base_models.append(('svm', SVC(random_state=42)))

meta_model = LogisticRegression(max_iter=10000)  # Increased max_iter for meta model as well

clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, n_jobs=-1)

clf.fit(X_train, y_train)

# Perform cross-validation with multiple scoring metrics
# scores = cross_val_score(clf, X, y, cv=5)
print(classification_report(y_val, clf.predict(X_val)))


# print(scores.mean())
# 83 min
# 0.4800848691221959
# 0.34316

              precision    recall  f1-score   support

           0       0.80      0.57      0.67        14
           1       0.40      0.29      0.33        21
           2       0.40      0.70      0.51       157
           3       0.32      0.19      0.24        31
           4       0.86      0.67      0.75        45
           5       0.20      0.14      0.17         7
           6       0.62      0.65      0.64        92
           7       0.50      0.27      0.35        45
           8       0.51      0.68      0.58       103
           9       0.73      0.78      0.75        67
          10       1.00      0.44      0.61        32
          11       0.79      0.65      0.71        46
          12       0.46      0.35      0.40        31
          13       0.22      0.11      0.15        37
          14       0.38      0.28      0.32        36
          15       0.34      0.43      0.38        51
          16       0.58      0.29      0.39        24
          17       0.33    

# Inference

In [58]:
# test 전처리가 끝난 상태로 저장됐기 때문에 바로 Inference 가능
try : test = test.drop(columns=['ID'])
except : pass

In [59]:
test.head()

,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,3,0,0,0,0,30,...,0,0,0,0,0,30,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
test_filled = test.fillna(0)

In [61]:
predictions = clf.predict(test_filled)

In [62]:
original_labels = le_subclass.inverse_transform(predictions)

# Submisson

In [64]:
submisson = pd.read_csv("./open/sample_submission.csv")

In [65]:
submisson["SUBCLASS"] = original_labels

In [66]:
submisson.to_csv('./submission/best.csv', encoding='UTF-8-sig', index=False)